In [25]:
import numpy as np
import matplotlib.pyplot as plt

from music import *
from matched_beamforming import *
from utils import *
from flexibeam import *

## Loading data

In [26]:
real_data_folder = "../data/"
real_data_1 = real_data_folder + "data_1.npz"
real_data_2 = real_data_folder + "data_2.npz"

In [27]:
def load_data(data_str):
    npz_data = np.load(data_str)
    return npz_data["XYZ"], npz_data["S"], float(npz_data["T_slot"]), float(npz_data["lambda_"]), npz_data["src_model"]

In [28]:
antenna_positions, cov, timeslot_durations, wavelength, transmitter_positions = load_data(real_data_1)

In [29]:
plt.plot(antenna_positions[:,0],
         antenna_positions[:,1],
         'bo',
         transmitter_positions[:,0],
         transmitter_positions[:,1],
         'ro'
         )
plt.show()

## MUSIC for DOA test

In [30]:
# Parameters for MUSIC

resolution = 0.15
time_t = 0

In [31]:
def plot_music_estimation(antenna_positions, cov, wavelength, transmitter_positions, nb_doas, time_t, resolution):
    doas, _, _, psd, thetas = music_algorithm(cov[time_t], antenna_positions, wavelength, nb_doas, resolution=resolution)
    thetas *= np.pi/180
    print(doas)
    plt.polar(list(map(cartesian_to_arg,transmitter_positions)), np.ones(len(transmitter_positions)), "go", thetas, psd/np.max(psd), '-k')
    plt.show()

In [32]:
antenna_positions, cov, timeslot_durations, wavelength, transmitter_positions = load_data(real_data_1)

plt.plot(antenna_positions[:,0],
         antenna_positions[:,1],
         'bo',
         transmitter_positions[:,0],
         transmitter_positions[:,1],
         'ro'
         )
plt.show()

print("Dataset I")
plot_music_estimation(antenna_positions, cov, wavelength, transmitter_positions, 13, time_t, resolution)

In [33]:
antenna_positions, cov, timeslot_durations, wavelength, transmitter_positions = load_data(real_data_2)
plt.plot(antenna_positions[:,0],
         antenna_positions[:,1],
         'bo',
         transmitter_positions[:,0],
         transmitter_positions[:,1],
         'ro'
         )
plt.show()
print("Dataset 2")
plot_music_estimation(antenna_positions, cov, wavelength, transmitter_positions, 7, time_t, resolution)

## Matched beamforming test

In [34]:
angle = 224.25
angle = angle * np.pi/180
r0 = np.array([np.cos(angle), np.sin(angle)])
b_gains, ts = matched_beam(antenna_positions, wavelength, r0)

In [35]:
plt.polar(ts, b_gains, '-k')
plt.show()

In [36]:
antenna_positions, cov, timeslot_durations, wavelength, transmitter_positions = load_data(real_data_1)
# 7271317.527537093 for dataset 1
# 6872221.912682234 for dataset 2

In [37]:
average_throughput(matched_beam, antenna_positions, wavelength, transmitter_positions)

In [38]:
antenna_positions, cov, timeslot_durations, wavelength, transmitter_positions = load_data(real_data_2)

In [39]:
average_throughput(matched_beam, antenna_positions, wavelength, transmitter_positions)

In [40]:
b_gains, ts = matched_beam(antenna_positions, wavelength, r0)
plt.polar(ts, b_gains, '-k')
plt.show()

In [41]:
average_throughput(matched_beam, antenna_positions, wavelength, transmitter_positions)

### Using MUSIC to estimate DOAs, dataset 2

In [42]:
doas, _, _, _, _ = music_algorithm(cov[time_t], antenna_positions, wavelength, 7, resolution=resolution)
doas = [np.array([np.cos(t), np.sin(t)]) for t in doas]

In [43]:
average_throughput(matched_beam, antenna_positions, wavelength, doas)